In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd.profiler as profiler

In [ ]:
# how many sequences are there in one batch
batch_size = 64
# how many characters does one training sequence have
block_size = 256
num_training_iterations = 5000
# after how many training iterations we want to run eval
eval_interval = 500
# how many batches we want to averge over for averaging loss during eval
eval_iters = 200
# learning rate passed into the optimizer
learning_rate = 3e-4
# embedding dimension
n_embd = 384
# number of heads
n_head = 6
# number of layers
n_layer = 6
# dropout ratio
dropout = 0.2
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
torch.manual_seed(1337)

In [ ]:
with open("shakespeare.txt", 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long, device=device)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]
print(len(data))

In [ ]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters, device=device)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None, kvcache=None):
        B, S = idx.shape
        # (B,S,H) every token in every batch gets embedded
        tok_emb = self.token_embedding_table(idx)
        # (S,H) every index of the sequence gets an embedding
        pos_emb = self.position_embedding_table(torch.arange(S, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B,T, vocab_size)

        if targets is None:
            loss =None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [ ]:
class Head(nn.Module):
    def __init__(self, head_size) -> None:
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
    
    @torch.compile
    def forward(self, x):
        B, S, H = x.shape
        k = self.key(x) # (BATCH,SEQUENCE,HIDDEN) -> (BATCH,SEQUENCE,HEAD)
        q = self.query(x) # (BATCH,SEQUENCE,HIDDEN) -> (BATCH,SEQUENCE,HEAD)
        wei = q @ k.transpose(-2, -1) * n_embd**-0.5 # (B,S,HEAD) @ (B,HEAD,S) -> (B,S,S)
        wei = wei.masked_fill(self.tril[:S, :S] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v # (B,S,S) @ (B,S,HEAD) -> (B,S,HEAD)
        return out

In [ ]:
class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size) -> None:
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head) -> None:
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:
model = BigramLanguageModel()
model = model.to(torch.float32).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
for iter in range(num_training_iterations):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    generated = model.generate(context, max_new_tokens=1)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated = model.generate(context, max_new_tokens=500)

print(decode(generated[0].tolist()))

In [ ]:
head_size = 16
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)
key = torch.nn.Linear(C, head_size, bias=False)
query = torch.nn.Linear(C, head_size, bias=False)
value = torch.nn.Linear(C, head_size, bias=False)
k = key(x)
q = query(x)
v = value(x)
wei = q @ k.transpose(-2, -1) * head_size**-0.5
wei.shape

In [ ]:
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = torch.nn.functional.softmax(wei, dim=-1)
out = wei @ v
out.shape

In [ ]:
def time_multihead_attention(input):
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    
    head_size = n_embd // n_head
    mha = MultiHeadAttention(n_head, head_size)

    for _ in range(5):
        mha(input)

    start.record()
    mha(input)
    end.record()
    torch.cuda.synchronize()
    return start.elapsed_time(end)

print(time_multihead_attention(torch.zeros((1,1,n_embd))))